Importing libraries

In [1]:
import pandas as pd
import numpy as np

Reading data

In [2]:
customers = pd.read_csv('olist_customers_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')

# Data preprocessing

Сheck for duplicates:

In [3]:
customers.shape[0] == customers.customer_id.nunique()

True

In [4]:
orders.shape[0] == orders.order_id.nunique()

True

Сombining dataframes for further analysis:

In [5]:
data = orders.merge(customers, on = 'customer_id')

In [6]:
data.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
customer_unique_id               object
customer_zip_code_prefix          int64
customer_city                    object
customer_state                   object
dtype: object

Replacing the data type with datetime for date columns:

In [7]:
data[['order_purchase_timestamp', 
      'order_approved_at', 
      'order_delivered_carrier_date', 
      'order_delivered_customer_date', 
      'order_estimated_delivery_date']] = data[['order_purchase_timestamp', 
                                                'order_approved_at', 
                                                'order_delivered_carrier_date', 
                                                'order_delivered_customer_date', 
                                                'order_estimated_delivery_date']].apply(pd.to_datetime)

We consider only orders with the "delivered" status to be purchases, since in all other cases orders can be canceled

In [8]:
d_data = data[data.order_status == "delivered"]

**1. How many users have made a purchase only once?**

In [9]:
(d_data.customer_unique_id.value_counts() == 1).sum()

90557

**2. How many orders per month on average are not delivered for various reasons (provide a breakdown by reasons)?**

We consider orders with the status "canceled" or "unavailable" to be undelivered:

In [10]:
not_delivered_1 = data[data.order_status == "canceled"]
not_delivered_2 = data[data.order_status == "unavailable"]

In [11]:
x = not_delivered_1.groupby(not_delivered_1.order_purchase_timestamp.dt.month) \
             .agg({'order_id': 'count'}) \
             .mean() \
             .round(0)
y = not_delivered_2.groupby(not_delivered_2.order_purchase_timestamp.dt.month) \
             .agg({'order_id': 'count'}) \
             .mean() \
             .round(0)

We calculate the average number of orders per month for each group

In [12]:
print('Average number of undelivered orders per month:', int((x+y).iloc[0]))
print('Average number of canceled orders per month:', int(x.iloc[0]))
print('Average number of unavailable orders per month:', int(y.iloc[0]))

Average number of undelivered orders per month: 103
Average number of canceled orders per month: 52
Average number of unavailable orders per month: 51


Сhecking the latest dataframe for duplicates

In [13]:
items.duplicated().sum()

0

Сombining dataframes for analysis

In [14]:
order_items = items.merge(orders, on='order_id')

In [15]:
order_items[['order_purchase_timestamp', 
             'order_approved_at', 
             'order_delivered_carrier_date', 
             'order_delivered_customer_date', 
             'order_estimated_delivery_date']] = order_items[['order_purchase_timestamp', 
                                                              'order_approved_at', 
                                                              'order_delivered_carrier_date', 
                                                              'order_delivered_customer_date', 
                                                              'order_estimated_delivery_date']].apply(pd.to_datetime)

Because we consider only delivered orders as purchases, we filter them by status for further analysis

In [16]:
d_order_items = order_items[order_items.order_status == "delivered"]

**3. For each product, determine which day of the week the product is most frequently purchased.**

First, let's count the number of purchases of each product by day of the week

In [17]:
amount = d_order_items.groupby([d_order_items.order_purchase_timestamp.dt.day_name(), d_order_items.product_id]) \
                 .order_item_id.count() \
                 .reset_index() \
                 .rename(columns={'order_purchase_timestamp' : 'weekday'})

In [18]:
amount.loc[amount.groupby('product_id').order_item_id.idxmax()] \
     .reset_index() \
     .drop(columns = 'index') \
     .rename(columns={'order_item_id' : 'quantity'})

,weekday,product_id,quantity
0,Sunday,00066f42aeeb9f3007548bb9d3f33c38,1
1,Tuesday,00088930e925c41fd95ebfe695fd2655,1
2,Thursday,0009406fd7479715e4bef61dd91f2462,1
3,Friday,000b8f95fcb9e0096488278317764d19,1
4,Tuesday,000d9be29b5207b54e86aa1b1ac54872,1
...,...,...,...
32211,Saturday,fff6177642830a9a94a0f2cba5e476d1,1
32212,Monday,fff81cc3158d2725c0655ab9ba0f712c,1
32213,Friday,fff9553ac224cec9d15d49f5a263411f,1
32214,Tuesday,fffdb2d0ec8d6a61f0a0a0db3f25b441,2


**4. How many purchases per week, on average, does each user make (per month)?**

First we count the number of purchases per month

In [19]:
purchases = d_data.groupby(['customer_unique_id', pd.Grouper(key='order_purchase_timestamp', freq='ME')]) \
                  .order_id.count() \
                  .reset_index()

After this, we create new columns and find the necessary data

In [20]:
purchases['weeksinmonth'] = purchases.order_purchase_timestamp.dt.daysinmonth / 7
purchases['weekly_purchases'] = purchases.order_id/purchases.weeksinmonth

In [21]:
purchases.groupby('customer_unique_id').weekly_purchases.mean()

customer_unique_id
0000366f3b9a7992bf8c76cfdf3221e2    0.225806
0000b849f77a49e4a4ce2b2a4ca5be3f    0.225806
0000f46a3911fa3c0805444483337064    0.225806
0000f6ccb0745a6a4b88665a16c9f078    0.225806
0004aac84e0df4da2b147fca70cf8255    0.233333
                                      ...   
fffcf5a5ff07b0908bd4e2dbc735a684    0.233333
fffea47cd6d3cc0a88bd621562a9d061    0.225806
ffff371b4d645b6ecea244b27531430a    0.250000
ffff5962728ec6157033ef9805bacc48    0.225806
ffffd2657e2aad2907e67c3e9daecbeb    0.225806
Name: weekly_purchases, Length: 93358, dtype: float64

**5. Cohort analysis of users**

First, find the date of first purchase for each user

In [22]:
first_purchase = d_data.groupby('customer_unique_id').order_purchase_timestamp.min().reset_index()

After this, we divide these dates by month, these will be the cohorts

In [23]:
first_purchase['cohort'] = first_purchase.order_purchase_timestamp.dt.to_period('M')
first_purchase = first_purchase.rename(columns={'order_purchase_timestamp' : 'first_purchase'})

Add new columns with the date of first purchase and cohort to the main dataframe using a join

In [24]:
d_data = d_data.merge(first_purchase, on='customer_unique_id')

Group by cohort and month of purchase and count the number of unique buyers for each group

In [25]:
cohort_data = d_data.groupby([d_data.cohort, d_data.order_purchase_timestamp.dt.to_period('M')]) \
                    .customer_unique_id.nunique() \
                    .reset_index()

In [26]:
cohort_data

,cohort,order_purchase_timestamp,customer_unique_id
0,2016-09,2016-09,1
1,2016-10,2016-10,262
2,2016-10,2017-04,1
3,2016-10,2017-07,1
4,2016-10,2017-09,1
...,...,...,...
214,2018-06,2018-07,25
215,2018-06,2018-08,16
216,2018-07,2018-07,5949
217,2018-07,2018-08,31


Create new columns to determine the number of months since the formation of the cohort and retention

In [27]:
cohort_data['month_number'] = cohort_data.order_purchase_timestamp.astype('int64') - cohort_data.cohort.astype('int64')
cohort_data['retention'] = cohort_data['customer_unique_id'] / cohort_data.groupby('cohort')['customer_unique_id'].transform('first')

Find the cohort with the highest retention in the 3rd month

In [28]:
cohort_data[cohort_data.month_number == 3].sort_values(by='retention', ascending=False).head(1)

,cohort,order_purchase_timestamp,customer_unique_id,month_number,retention
102,2017-06,2017-09,13,3,0.004281


**6. RFM segmentation**

To start doing segmentation we need to join all the tables. We take a ready-made table with customers and delivered orders, paste information on the goods.

In [29]:
all_data = d_data.merge(items, on='order_id')

We calculate the price of orders

In [30]:
order_price = all_data.groupby(['order_id', 'customer_unique_id', 'order_purchase_timestamp']).price.sum().reset_index()

Since the data is old, we will assume that “now” is the last date of purchase + 1 day

In [31]:
NOW = order_price.order_purchase_timestamp.max() + pd.Timedelta(days=1)
NOW

Timestamp('2018-08-30 15:00:37')

Create a column with the difference in days between “now” and the date of purchase

In [32]:
order_price['days'] = (NOW - order_price['order_purchase_timestamp']).dt.days

Create a new dataframe: group by users, then calculate Recency, Frequency and Monetary.

Recency - number of days since last purchase, Frequency - number of purchases for all time, Monetary - amount of money for all time

In [33]:
rfm = order_price.groupby('customer_unique_id', as_index = False) \
                .agg({'days': 'min', 'order_id': 'count', 'price': 'sum'}) \
                .rename(columns={'days':'Recency', 'order_id': 'Frequency', 'price': 'Monetary'})

Divide our data into segments. Since we have a very small number of repeat purchases (>1), we take 2 groups for Frequency. For other characteristics there are 4 groups each.

In [34]:
quintiles_1 = rfm[['Recency']].quantile([.25, .5, .75]).to_dict()
quintiles_3 = rfm[['Monetary']].quantile([.25, .5, .75]).to_dict()

In [35]:
def r_score(x):
    if x <= quintiles_1['Recency'][.25]:
        return 4
    elif x <= quintiles_1['Recency'][.5]:
        return 3
    elif x <= quintiles_1['Recency'][.75]:
        return 2
    else:
        return 1

def f_score(x):
    if x <= 1:
        return 1
    else:
        return 4  

def m_score(x):
    if x <= quintiles_3['Monetary'][.25]:
        return 1
    elif x <= quintiles_3['Monetary'][.5]:
        return 2
    elif x <= quintiles_3['Monetary'][.75]:
        return 3
    else:
        return 4

 We use functions to assign the required speed and create a new column. Because there are only 2 groups in Frequency, F is either 1 (less than one purchase) or 4 (more than one purchase)

In [36]:
rfm['R'] = rfm['Recency'].apply(r_score)
rfm['F'] = rfm['Frequency'].apply(f_score)
rfm['M'] = rfm['Monetary'].apply(m_score)
rfm['RFM Score'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)

We make a dictionary in which we denote the boundaries of each segment. Let's take the segments from the example adjusted for our data as a sample. Segment boundaries can be seen from the dictionary structure and quantile values.

In [37]:
segt_map = {
    r'11[1-4]': 'hibernating',
    r'14[1-2]': 'at risk',
    r'14[3-4]': 'can\'t loose',
    r'21[1-3]': 'about to sleep',
    r'2[1-4][1-4]': 'need attention',
    r'[3-4]4[3-4]': 'loyal customers',
    r'[3-4]1[3-4]': 'promising',
    r'[3-4]1[1-2]': 'new customers',
    r'[3-4]4[1-2]': 'potential loyalists'
}

# hibernating - the last purchase was a long time ago, a small number of purchases may no longer be active
# at risk - the last purchase was also a long time ago, but there was more than one purchase
# can't loose - the last purchase was a long time ago, but there was more than one purchase and the client is in the top for money spent
# about to sleep - haven't made purchases for a long time
# need attention - there have been no purchases for a long time, while clients from this group either bought often or were in the top for money spent
# loyal customers - our best customers, who shop regularly and spend the most
# promising - recently made a purchase, spend above average
# new customers - recently made a purchase for a small amount
# potential loyalists - recently made several purchases, but not for a very large amount


rfm['Segment'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(segt_map, regex=True)
rfm.Segment.value_counts()

Segment
new customers          22938
hibernating            22732
promising              22360
about to sleep         17125
need attention          6075
loyal customers         1316
can't loose              463
potential loyalists      233
at risk                  116
Name: count, dtype: int64